# Learn Selenium Scrape on LinkedIn Job Post
References: Web Scraping LinkedIn Job Page with Selenium Python _ by Mert Kucukkuru _ Medium

## Overview
It is a notebook file that stores dirty code for the project.
- Start the project by installing the required library (selenium), and setting up the web driver (chrome web driver).
- Create a .txt file that store your username & LinkedIn password.
- The code will scrape the job offers and their info by visiting each offer scraped on every page.
- Store the job offer in a .csv file and the description in a .txt file.

### The whole trial code

In [2]:
import time
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

## PART I
# setup the path for the webdriver
path = 'C:/Users/User/Downloads/chromedriver.exe'
driver = webdriver.Chrome(path)

# maximizing the window
driver.maximize_window()
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

# entering the site 
driver.get('https://www.linkedin.com/login')
time.sleep(2) # made the program wait 2 sec to be sure that the page is loaded

# user credentials
# reading txt file store your account id and password
with open('user_credentials.txt', 'r',encoding='utf-8') as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]

user_name = user_credentials[0] # your account id
password = user_credentials[1] # your account password

driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(user_name)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(password)
time.sleep(2)

# login button
driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(20)

# job page
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)

# go to search result
search_keyword = 'data science'
search_loc = 'indonesia'
driver.find_element(By.XPATH,'//*[@id="global-nav-search"]/div/div[2]/div[2]').click()
driver.find_element(By.XPATH, '//*[@id="jobs-search-box-keyword-id-ember249"]').send_keys(search_keyword)
driver.find_element(By.XPATH, '//*[@id="jobs-search-box-location-id-ember249"]').send_keys(search_loc + '\n')
time.sleep(2)


## PART II 
# get all links for the offers
links = []

print('Links are being collected now.')
try:
    # navigate 3 pages
    for page in range (2,4): 
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'scaffold-layout__list-container')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith('https://www.linkedin.com/jobs/view') and a.get_attribute('href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down
            driver.execute_script('arguments[0].scrollIntoView();', job)

        print(f'Collecting the links in the page : {page-1}')
        # go to next page through loop
        driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)
except:
    pass
print('Found' + str(len(links)) + 'links for job offers')

# try and except: pass block in case there is a missing element in a page
# it prevents automation from failing and stopping the whole process instead of passing the problematic 1 specific page 


## PART III
# create empty list to store the info
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = []
job_desc = []

i = 0 ; j = 1

# visit each link one by one to scrape the info
print('Visiting the links and collecting info just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(3)
        # click 'see more'
        driver.find_element(By.CLASS_NAME, 'artdeco-card__actions').click()
        time.sleep(2)
    except:
        pass

    # find the general information from the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME,'h1').text)
            company_names.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__company-name').text)
            company_locations.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__bullet').text)
            work_methods.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__workplace-type').text)
            post_dates.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__posted-date').text)
            work_times.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__job-insight').text)
            print(f'Scraping the Job Offer {j} DONE.')
            j+=1
        except:
            pass
        time.sleep(2)

    # scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME,'jobs-box__html-content').text
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')
        time.sleep(4)


# PART IV
# creating the dataframe
df = (pd.DataFrame(list(zip(job_titles,company_names,company_locations,
                            work_methods,post_dates,work_times)),
                            columns=['job_title','company_name','company_location',
                                    'work_method','post_date','work_time']))

# storing into csv file
df.to_csv('job_offers.csv', index=False)

# pull out the job description as txt file
with open('job_descriptions.txt', 'w',encoding='utf-8') as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')

C:\Users\User\AppData\Local\Temp\ipykernel_7576\2280551933.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [ ]:
# preview the data
datas = pd.read_csv('job_offers.csv')
datas

### The dirty notes

In [26]:
import time
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

## PART I
# setup the path for the webdriver
# driver path

path = 'C:/Users/User/Downloads/chromedriver.exe'
driver = webdriver.Chrome(path)

# maximizing the window
driver.maximize_window()
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

# entering the site 
driver.get('https://www.linkedin.com/login')
time.sleep(2) # it will make the program wait 2 sec to be sure that the page is loaded

# user credentials
# reading txt file, storing your account id pass

with open('user_credentials.txt', 'r',encoding='utf-8') as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]

user_name = user_credentials[0] # 1st 
password = user_credentials[1] # 2nd

driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(user_name)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(password)
time.sleep(2)

# login button
driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(20)

# job page
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)

# goto search result directly via link
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3326797135&geoId=102478259&keywords=data%20science&location=Indonesia&refresh=true')
time.sleep(1)
#driver.find_element('xpath','//*[@id="jobs-search-box-keyword-id-ember250"]').click()
#search_keyword = 'data science'
#search_loc = 'indonesia'
#driver.find_element('xpath', '//*[@id="jobs-search-box-keyword-id-ember250"]').send_keys(search_keyword)
#driver.find_element('xpath', '//*[@id="jobs-search-box-keyword-id-ember250"]').send_keys(search_loc).sendKeys(Keys.ENTER)


## PART II 
# get all links for these offers
links = []
# navigate 2 pages
print('Links are being collected now.')
try:
    for page in range (2,3):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'scaffold-layout__list-container')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith('https://www.linkedin.com/jobs/view') and a.get_attribute('href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down
            driver.execute_script('arguments[0].scrollIntoView();', job)

        print(f'Collecting the links in the page : {page-1}')
        # go to next page
        driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)
except:
    pass
print('Found' + str(len(links)) + 'links for job offers')


## PART III
# create empty list to store the info
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = []
job_desc = []

i = 0 ; j = 1

# visit each link one by one to scrape the info
print('Visiting the links and collecting info just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(3)
        # click 'see more'
        driver.find_element(By.CLASS_NAME, 'artdeco-card__actions').click()
        time.sleep(2)
    except:
        pass

    # find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME,'h1').text)
            company_names.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__company-name').text)
            company_locations.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__bullet').text)
            work_methods.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__workplace-type').text)
            post_dates.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__posted-date').text)
            work_times.append(content.find_element(By.CLASS_NAME,'jobs-unified-top-card__job-insight').text)
            print(f'Scraping the Job Offer {j} DONE.')
            j+=1
        except:
            pass
        time.sleep(2)

    # scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME,'jobs-box__html-content').text
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')
        time.sleep(4)

# PART IV
# creating the dataframe
df = (pd.DataFrame(list(zip(job_titles,company_names,company_locations,
                            work_methods,post_dates,work_times)),
                            columns=['job_title','company_name','company_location',
                                    'work_method','post_date','work_time']))

# storing into csv file
df.to_csv('job_offers.csv', index=False)

# pull out the job description as txt file
with open('job_descriptions.txt', 'w',encoding='utf-8') as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')


C:\Users\User\AppData\Local\Temp\ipykernel_8488\2853580805.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Links are being collected now.
Found25links for job offers
Visiting the links and collecting info just started.
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8
Scraping the Job Offer 8
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 16
Scraping t

In [27]:
datas = pd.read_csv('job_offers.csv')
datas

,job_title,company_name,company_location,work_method,post_date,work_time
0,Data Scientist Intern,Sribuu (YC W22),"Jakarta, Indonesia",Remote,4 days ago,Internship
1,Jr Data Scientist,PT Imani Prima,"Jakarta, Jakarta, Indonesia",On-site,4 weeks ago,Full-time · Associate
2,AI Research Scientist,PT Bank Rakyat Indonesia (Persero) Tbk,"Jakarta, Indonesia",On-site,2 days ago,Full-time · Mid-Senior level
3,Data Engineer Intern,dentsu international,"Jakarta, Jakarta, Indonesia",On-site,3 weeks ago,Full-time · Internship
4,Business Analytics,Citi,"Jakarta, Jakarta, Indonesia",On-site,1 day ago,Full-time
5,Client Engineering - Data Scientist,IBM,"Jakarta, Jakarta, Indonesia",On-site,1 week ago,Full-time
6,Data Analyst Lead,PT Alto Network,"Jakarta, Jakarta, Indonesia",On-site,1 day ago,Full-time · Mid-Senior level
7,Data Engineer,Binar Academy,"Tangerang, Banten, Indonesia",On-site,1 week ago,Full-time
8,Client Engineering - Data Scientist,IBM,"Jakarta, Jakarta, Indonesia",On-site,20 hours ago,"IDR7,000,000/month - IDR15,000,000/month · Ful..."
9,Data Engineer,Silvertech Asia,"Jakarta, Jakarta, Indonesia",On-site,2 days ago,Full-time
